# create the data frames, about the fruits with the points


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import shutil

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  +
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Fruits')

DATA_PATH2 = os.path.join('Fruits-model')

# Actions that we try to detect
actions = np.array(['Banano', 'Coco', 'Durazno', 'Manzana', 'Piña'])
########### Este listas de ALS, cambiar LSC #########

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 25

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Guardar - primero video como clave puntos  

In [17]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):#start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# Creating the link between the folder

In [1]:
import os
import shutil
import numpy as np

def link_dataset(DATA_PATH, DATA_PATH2):
    """
    Une el contenido del dataset en DATA_PATH2 al dataset en DATA_PATH.
   
    Args:
    DATA_PATH (str): Ruta al directorio del dataset principal.
    DATA_PATH2 (str): Ruta al directorio del dataset a unir.
   
    Returns:
    None
    """
    if not os.path.exists(DATA_PATH) or not os.path.exists(DATA_PATH2):
        raise ValueError(f"Uno o ambos directorios no existen: {DATA_PATH}, {DATA_PATH2}")

    for action in os.listdir(DATA_PATH2):
        rute_first = os.path.join(DATA_PATH, action)
        rute_second = os.path.join(DATA_PATH2, action)

        if not os.path.exists(rute_first):
            # Copiar todo el árbol si no existe
            shutil.copytree(rute_second, rute_first)
        else:
            for sequential in os.listdir(rute_second):
                rute_sequence_2 = os.path.join(rute_second, sequential)
                rute_sequence_1 = os.path.join(rute_first, sequential)

                if not os.path.exists(rute_sequence_1):
                    # Copiar toda la secuencia si no existe
                    shutil.copytree(rute_sequence_2, rute_sequence_1)
                else:
                    # Contar frames existentes
                    existing_frames = len(os.listdir(rute_sequence_1))
                    new_frames = len(os.listdir(rute_sequence_2))

                    # Si hay menos frames en el destino que en la fuente, copia los nuevos
                    if existing_frames < new_frames:
                        for frame in os.listdir(rute_sequence_2):
                            shutil.copy2(os.path.join(rute_sequence_2, frame), rute_sequence_1)

                    print(f"Frames en '{rute_sequence_1}': {existing_frames}, nuevos frames: {new_frames}")

    print(f"Dataset dos vinculado exitosamente al dataset uno en {DATA_PATH}")

if __name__ == "__main__":
    DATA_PATH = r"C:\Users\fabri\Documents\GitHub\linkproject_ia\Fruits"
    DATA_PATH2 = r"C:\Users\fabri\Documents\GitHub\linkproject_ia\fruit-model"
    link_dataset(DATA_PATH, DATA_PATH2)


Frames en 'C:\Users\fabri\Documents\GitHub\linkproject_ia\Fruits\banano\0': 30, nuevos frames: 30
Frames en 'C:\Users\fabri\Documents\GitHub\linkproject_ia\Fruits\banano\1': 30, nuevos frames: 30
Frames en 'C:\Users\fabri\Documents\GitHub\linkproject_ia\Fruits\banano\10': 30, nuevos frames: 30
Frames en 'C:\Users\fabri\Documents\GitHub\linkproject_ia\Fruits\banano\11': 30, nuevos frames: 30
Frames en 'C:\Users\fabri\Documents\GitHub\linkproject_ia\Fruits\banano\12': 30, nuevos frames: 30
Frames en 'C:\Users\fabri\Documents\GitHub\linkproject_ia\Fruits\banano\13': 30, nuevos frames: 30
Frames en 'C:\Users\fabri\Documents\GitHub\linkproject_ia\Fruits\banano\14': 30, nuevos frames: 30
Frames en 'C:\Users\fabri\Documents\GitHub\linkproject_ia\Fruits\banano\15': 30, nuevos frames: 30
Frames en 'C:\Users\fabri\Documents\GitHub\linkproject_ia\Fruits\banano\16': 30, nuevos frames: 30
Frames en 'C:\Users\fabri\Documents\GitHub\linkproject_ia\Fruits\banano\17': 30, nuevos frames: 30
Frames en 'C